In [1]:
import requests
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
#creating an model object
model = ChatOpenAI()
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020B878AB8E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020B8A80FCD0>, root_client=<openai.OpenAI object at 0x0000020B878A8E20>, root_async_client=<openai.AsyncOpenAI object at 0x0000020B8A80FC40>, model_kwargs={}, openai_api_key=SecretStr('**********'))

# step:1 creating Tools by using tool decorator

In [3]:
from langchain.tools import tool
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str,target_currency:str) -> float:
    """this function will return conversion factor between given base currency and target currency"""
    url = f"https://v6.exchangerate-api.com/v6/33aec66e25e1df4937114926/pair/{base_currency}/{target_currency}"
    
    #now sending the request to this URL.
    response = requests.get(url=url)
    
    if response.status_code == 200 :
        return response.json()
    else:
        return "Wrong API KEY"


# result = get_conversion_factor.invoke({'base_currency':"USD",'target_currency':"INR"})
# result

In [4]:
# result["conversion_rate"]

```
InjectedToolArg means do not fill these argument i developer will inject this value after running earlier tools
```

In [5]:
#creating another who perform multiplication btn base_currency and conversion factor.

@tool
def convert(base_currency_value:float,conversion_factor:Annotated[float,InjectedToolArg]) ->float:
    """given a currency conversion rate this function calculate target currency value from given base currency value"""
    
    return base_currency_value* conversion_factor

In [6]:
#convert.invoke({'base_currency_value':10,'conversion_factor':result["conversion_rate"]})

# step:2 now binding the tool with llm

In [7]:
llm_with_tool = model.bind_tools([get_conversion_factor,convert])
llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020B878AB8E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020B8A80FCD0>, root_client=<openai.OpenAI object at 0x0000020B878A8E20>, root_async_client=<openai.AsyncOpenAI object at 0x0000020B8A80FC40>, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'this function will return conversion factor between given base currency and target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculate target currency value from given base currency value', 'parameters': {'propert

# query forming

In [8]:
message = [
    HumanMessage(content="What is the conversion factor between USD and INR, and based on that can you convert 6 usd to inr")
]

# step:3 calling the tools

In [9]:
ai_msg = llm_with_tool.invoke(message)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XWQyhHE7y7qneam95NLV1mE8', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_gRbAvCaXJxc3XVA7PuJKcwk6', 'function': {'arguments': '{"base_currency_value": 6}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 117, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAHmqlZQEKQNPCKXnZb9oDpzhJVcB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--19b51c2d-9b9c-4185-986c-2d6be1f5ed59-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_curr

In [10]:
ai_msg.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_XWQyhHE7y7qneam95NLV1mE8',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 6},
  'id': 'call_gRbAvCaXJxc3XVA7PuJKcwk6',
  'type': 'tool_call'}]

In [11]:
message.append(ai_msg)

In [12]:
message

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 6 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XWQyhHE7y7qneam95NLV1mE8', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_gRbAvCaXJxc3XVA7PuJKcwk6', 'function': {'arguments': '{"base_currency_value": 6}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 117, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAHmqlZQEKQNPCKXnZb9oDpzhJVcB', 'service_tier': 'defaul

# iterating the message

In [13]:
import json
for tool_call in ai_msg.tool_calls:
    #executing the 1st tools and get the conversion rate value.
    if tool_call["name"] == "get_conversion_factor":
        tool_msg1 = get_conversion_factor.invoke(tool_call)
        #print(tool_msg1)
    
        #fetching the conversion rate.
        conversion_rate = json.loads(tool_msg1.content)["conversion_rate"]
        
        #appending the tool msg to lst object
        message.append(tool_msg1)
    
    
        
    #then execute the second tool using the conversion rate from tool 1.
    if tool_call["name"] == "convert":
        #fetch the current argument and ingesting with tool1 answer
        tool_call['args']['conversion_factor'] = conversion_rate
        
        #calling tool2 
        tool_msg2 = convert.invoke(tool_call)
        
        #appending the tool msg to lst object
        message.append(tool_msg2)
        
        
    
message

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 6 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XWQyhHE7y7qneam95NLV1mE8', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_gRbAvCaXJxc3XVA7PuJKcwk6', 'function': {'arguments': '{"base_currency_value": 6}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 117, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAHmqlZQEKQNPCKXnZb9oDpzhJVcB', 'service_tier': 'defaul

# finally sending this msg to llm to get response

In [15]:
response = llm_with_tool.invoke(message)
response

AIMessage(content='The conversion factor between USD and INR is 88.0985. \n\nBased on this conversion factor, 6 USD is equivalent to approximately 528.59 INR.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 328, 'total_tokens': 365, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAHqHOA3LugFnQ0HROUURzc9AkWeE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1b91fe6-140d-4fb0-abdb-1d27955c1785-0', usage_metadata={'input_tokens': 328, 'output_tokens': 37, 'total_tokens': 365, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [16]:
response.content

'The conversion factor between USD and INR is 88.0985. \n\nBased on this conversion factor, 6 USD is equivalent to approximately 528.59 INR.'